In [1]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Fri Jun 10 20:00:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 33%   42C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 36%   

# MongoDB

In [4]:
from pymongo import MongoClient
import pymongo

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2013 = db.rm_patent_cn_2013
    db_rm_us_2013 = db.rm_patent_us_2013
    db_rm_de_2013 = db.rm_patent_de_2013

    
except errors.ConnectionFailure as err:
    print(err)

# 2013

## Company數量

In [6]:
data_assignee = db_rm_de_2013.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [8]:
data_assignee = db_rm_cn_2013.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_us_2013.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [11]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
257,Jiangsu IoT Research and Development Center,63
305,Nanjing IoT Sensor Technology Co Ltd,55
109,Cisco Technology Inc,38
71,Chengdu Qinchuan IoT Technology Co Ltd,34
104,China core Microelectronics Technology Chengdu...,22
473,The reservoir Hengke IOT System Co Ltd,21
416,Shanghai Nenggan Epc System Network Co Ltd,16
348,Qualcomm Inc,14
101,China United Creation (fujian) Internet Of Thi...,14
274,LG Electronics Inc,13


In [12]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [13]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Jiangsu IoT Research and Development Center,63,4.143135
1,Nanjing IoT Sensor Technology Co Ltd,55,4.007333
2,Cisco Technology Inc,38,3.637586
3,Chengdu Qinchuan IoT Technology Co Ltd,34,3.526361
4,China core Microelectronics Technology Chengdu...,22,3.091042
5,The reservoir Hengke IOT System Co Ltd,21,3.044522
6,Shanghai Nenggan Epc System Network Co Ltd,16,2.772589
7,Qualcomm Inc,14,2.639057
8,China United Creation (fujian) Internet Of Thi...,14,2.639057
9,LG Electronics Inc,13,2.564949


公司名稱正規化

In [14]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Co. Ltd.","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("..","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba")\
                                                    .replace("ZTE Intelligent IoT Technology","ZTE"))

In [15]:
com_df

,company,count,count_log
0,Jiangsu IoT Research and Development Center,63,4.143135
1,Nanjing IoT Sensor Technology,55,4.007333
2,Cisco Technology,38,3.637586
3,Chengdu Qinchuan IoT Technology,34,3.526361
4,China core Microelectronics Technology Chengdu...,22,3.091042
5,The reservoir Hengke IOT System,21,3.044522
6,Shanghai Nenggan Epc System Network,16,2.772589
7,Qualcomm,14,2.639057
8,China United Creation (fujian) Internet Of Thi...,14,2.639057
9,LG Electronics,13,2.564949


In [16]:
com_ls = list(com_df.company)

# Data 2013

富比士2000資料

In [17]:
f_data = pd.read_csv("./Forbes/Forbes_2013.csv")

In [18]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Industry,Country,Market Value,Sales,Profits,Assets,Rank,Forbes Webpage,Profits as % of Assets,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
20,Samsung Electronics,Semiconductors,South Korea,174.4,187.8,21.7,196.3,20,http://www.forbes.com/companies/samsung-electr...,0.110545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,Intel,Semiconductors,United States,105.7,53.3,11.0,84.4,77,http://www.forbes.com/companies/intel/,0.130332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,Qualcomm,Semiconductors,United States,111.6,20.5,6.6,44.8,164,http://www.forbes.com/companies/qualcomm/,0.147321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,Taiwan Semiconductor,Semiconductors,Taiwan,89.9,17.4,5.7,32.7,227,http://www.forbes.com/companies/taiwan-semicon...,0.174312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,Texas Instruments,Semiconductors,United States,38.8,12.8,1.8,20.0,408,http://www.forbes.com/companies/texas-instrume...,0.090000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,ASML Holding,Semiconductors,Netherlands,28.9,6.2,1.5,9.7,743,http://www.forbes.com/companies/asml-holding/,0.154639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
786,Broadcom,Semiconductors,United States,19.7,8.0,0.7,11.2,787,http://www.forbes.com/companies/broadcom/,0.062500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950,SK Hynix,Semiconductors,South Korea,18.0,9.6,-0.1,17.1,951,http://www.forbes.com/companies/sk-hynix/,-0.005848,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,Applied Materials,Semiconductors,United States,16.0,8.1,0.0,11.7,1143,http://www.forbes.com/companies/applied-materi...,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1149,Tokyo Electron,Semiconductors,Japan,8.5,7.6,0.4,9.3,1149,http://www.forbes.com/companies/tokyo-electron/,0.043011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [20]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [21]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
7,Qualcomm,14,2.639057
25,Intel,6,1.791759


In [25]:
# pbc 0.1, 0.1
comp_ls = ["Qualcomm","Telefonaktiebolaget LM Ericsson AB","ZTE","Shanghai Nenggan Epc System Network","Shenzhen Shenglu Iot Communication Technology","National Center for Advanced Packaging","Nanjing IoT Sensor Technology","LG Electronics"]

In [26]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [27]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
1,Nanjing IoT Sensor Technology,55,4.007333
6,Shanghai Nenggan Epc System Network,16,2.772589
7,Qualcomm,14,2.639057
9,LG Electronics,13,2.564949
11,Shenzhen Shenglu Iot Communication Technology,12,2.484907
12,ZTE,12,2.484907
13,Telefonaktiebolaget LM Ericsson AB,11,2.397895
15,National Center for Advanced Packaging,9,2.197225
24,ZTE,7,1.945910


In [28]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.014651979944181296
ari:  0.03737398573887386


## 使用Forbes做join

In [32]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [33]:
# pbc 0.1, 0.1
comp_ls = ["Qualcomm","Telefonaktiebolaget LM Ericsson AB","ZTE","Shanghai Nenggan Epc System Network","Shenzhen Shenglu Iot Communication Technology","National Center for Advanced Packaging","Nanjing IoT Sensor Technology","LG Electronics"]

In [34]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [35]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.032076985139240614
ari:  0.06802160399257595
